# monad-bayes example

In [33]:
-- A toy model for dice rolling from http://dl.acm.org/citation.cfm?id=2804317
-- Exact results can be obtained using Dist monad

import Control.Monad (liftM2)
import Control.Monad.IO.Class (liftIO)
import Control.Monad.Bayes.Class
import Control.Monad.Bayes.Sampler
import Control.Monad.Bayes.Population
import Control.Monad.Bayes.Inference.SMC

-- | A toss of a six-sided die.
die :: MonadSample m => m Int
die = uniformD [1..6]

-- | A sum of outcomes of n independent tosses of six-sided dice.
dice :: MonadSample m => Int -> m Int
dice 1 = die
dice n = liftM2 (+) die (dice (n-1))

-- | Toss of two dice where the output is greater than 4.
dice_hard :: MonadInfer m => m Int
dice_hard = do
  result <- dice 2
  condition (result > 4)
  return result

In [36]:
sampleIOfixed $ do
  res <- runPopulation $ smcMultinomial 0 10 dice_hard
  liftIO $ putStr $ unlines $ map show res

(3,0.0)
(7,9.999999999999998e-2)
(10,9.999999999999998e-2)
(6,9.999999999999998e-2)
(7,9.999999999999998e-2)
(9,9.999999999999998e-2)
(5,9.999999999999998e-2)
(7,9.999999999999998e-2)
(6,9.999999999999998e-2)
(8,9.999999999999998e-2)